In [1]:
import importlib
import data
from gurobipy import *

# get data from excel
dataframes = data.get_all_pandas()

LOADING DATA: 8it [00:40,  3.44s/it]                       

In [2]:
##### Respect du nombre de voies de chantier #####
def retrieve_occupations(OCCUPATIONS):
    # print(len(OCCUPATIONS))
    # print(OCCUPATIONS[0])
    # print(type(OCCUPATIONS[0]))
    # print(OCCUPATIONS[0].getValue())
    chantiers_df = dataframes["chantiers_df"]
    L = {}
    occupation_limits = [chantiers_df[chantiers_df["Chantier"]=="WPY_REC"]["Nombre de voies"].iloc[0],
            chantiers_df[chantiers_df["Chantier"]=="WPY_FOR"]["Nombre de voies"].iloc[0],
            chantiers_df[chantiers_df["Chantier"]=="WPY_DEP"]["Nombre de voies"].iloc[0]]
    for tache_name, sillon_id, line_expr in OCCUPATIONS["DEB"]:
        v = line_expr.getValue()
        
        if v == occupation_limits[0]:
            print("DEB limite atteinte")
        # if v > occupation_limits[0]:
        #     print("DEB limite DEPASSEE")
            
        L[tache_name, sillon_id] = v
        
    for tache_name, sillon_id, line_expr in OCCUPATIONS["FOR"]:
        v = line_expr.getValue()
        # if v == occupation_limits[1]:
        #     print("FOR limite atteinte")
        if v > occupation_limits[1]:
            print("FOR limite DEPASSEE")
        L[tache_name, sillon_id] = v


    for tache_name, sillon_id, line_expr in OCCUPATIONS["DEG"]:
        v = line_expr.getValue()
        # if v == occupation_limits[2]:
        #     print("DEG limite atteinte")
        if v > occupation_limits[2]:
            print("DEG limite DEPASSEE")
        L[tache_name, sillon_id] = v
    return L,occupation_limits


In [3]:
from openpyxl import Workbook
from datetime import datetime
from utils import min_to_jour

def show_solution(model, OCCUPATIONS):
    # print(var_dict)
    print("Objectif minimal: ", m.objVal)
    #build excel of tasks
    workbook = Workbook()
    sheet = workbook.active
    sillons_df = dataframes["sillons_df"]
    taches_df = dataframes["taches_df"]
    machines_df = dataframes["machines_df"]
    L, occupation_limits = retrieve_occupations(OCCUPATIONS)
    occupation_keys = L.keys()

    sheet["A1"] = "Id_tache"
    sheet["B1"] = "Type de tache"
    sheet["C1"] = "Jour"
    sheet["D1"] = "HDébut"
    sheet["E1"] = "Durée"
    sheet["F1"] = "Sillon"
    sheet["G1"] = "Minute"
    sheet["H1"] = "Occupation"
    row = 2
    for tache in var_dict.keys():
        for sillon_id in var_dict[tache].keys():
            sheet[f"B{row}"] = tache
            sheet[f"A{row}"] = tache + "_" + sillon_id
            sheet[f"C{row}"] = min_to_jour(var_dict[tache][sillon_id].X)[0]
            sheet[f"D{row}"] = min_to_jour(var_dict[tache][sillon_id].X)[1]
            if tache in list(machines_df["Machine"]):
                sheet[f"E{row}"] = machines_df[machines_df["Machine"] == tache]["Duree "].iloc[0]
            elif tache in list(taches_df["Type de tache humaine"]): 
                sheet[f"E{row}"] = taches_df[taches_df["Type de tache humaine"] == tache]["Durée"].iloc[0]
            sheet[f"F{row}"] = sillon_id
            sheet[f"G{row}"] = var_dict[tache][sillon_id].X
            if (tache, sillon_id) in occupation_keys:
                sheet[f"H{row}"] = L[tache, sillon_id]
            row += 1
    ##ocupation
    
    occupation_sheet = workbook.create_sheet("Occupation")
    sheet = occupation_sheet
    sheet["A1"] = "Mesure"
    sheet["B1"] = "WPY_REC"
    sheet["C1"] = "WPY_FOR"
    sheet["D1"] = "WPY_DEP"
    
    
    sheet["A2"] = "Taux max occupation"
    sheet["A3"] = "Nb max occupation"
    sheet["A4"] = "Nb max par chantier"
    
    chantier = "WPY_REC"
    occupation_max = max([L["arrivée Reception", sillon_id_key] for tache_name_key, sillon_id_key in L.keys() if tache_name_key == "arrivée Reception" ])
    capacity = occupation_limits[0]
    sheet["B2"] = 100*occupation_max/capacity
    sheet["B3"] = occupation_max
    sheet["B4"] = capacity
    
    chantier = "WPY_FOR"
    occupation_max = max([L["débranchement", sillon_id_key] for tache_name_key, sillon_id_key in L.keys() if tache_name_key == "débranchement" ])
    capacity = occupation_limits[1]
    sheet["C2"] = 100*occupation_max/capacity
    sheet["C3"] = occupation_max
    sheet["C4"] = capacity
    
    chantier = "WPY_DEP"
    
    occupation_max =max([L["dégarage / bouger de rame", sillon_id_key] for tache_name_key, sillon_id_key in L.keys() if tache_name_key == "dégarage / bouger de rame" ])
    capacity = occupation_limits[2]
    sheet["D2"] = 100*occupation_max/capacity
    sheet["D3"] = occupation_max
    sheet["D4"] = capacity
    
    
    currentDateAndTime = datetime.now().strftime("%d-%m_%H-%M-%S")
    workbook.save(filename=f"results/{data.NAME}-{currentDateAndTime}.xlsx")
# show_solution(m, OCCUPATIONS)


In [4]:
import variables
import contraintes
import importlib
import utils
importlib.reload(variables) #to remive in final .py

importlib.reload(utils)

<module 'utils' from 'c:\\Users\\remgo\\Desktop\\ProjetST7\\git\\utils.py'>

In [5]:
# create model
importlib.reload(contraintes) #to remive in final .py
m = Model()

# generate variables (integers)
var_dict = variables.generate_variablesJ1(m, dataframes)
# generate variables
B, OCCUPATIONS, max_horizon = contraintes.generate_contraintes(m, dataframes, var_dict)

final_train_task = dataframes["sillons_df"].copy()
final_train_task = final_train_task[final_train_task["LDEP"]=="WPY"]
final_train_task = final_train_task[final_train_task["JARR"]==final_train_task["JARR"].max()]
# # mono objo
# final_train_task = final_train_task[final_train_task["HDEP"]==final_train_task["HDEP"].max()]["train_id"].iloc[0]
# final_train_task = var_dict["essai de frein départ"][final_train_task]
# # multi objo
# final_train_task = quicksum([var_dict["essai de frein départ"][temp] for temp in list(final_train_task["train_id"])])
print(final_train_task)
m.setObjective(final_train_task, GRB.MINIMIZE)
# autre idée : maximiser la moyenne de l'ecart entre frein de depart et HDEP
# autre idée . minimiser l'horizon 
# m.setObjective(max_horizon, GRB.MINIMIZE)
# aute idee minimiser occupation
m.update()
# m.printStats()
# m.write("genconstr.mps")
# m.write("test.lp")
m.optimize()
# m.write("test.lp")
## !!!!! UNCOMMENT CONTRAINTE TACHE HUMAINE IN CONTRAINTES.PY
print("max horizon: ", max_horizon.X)
show_solution(m, OCCUPATIONS)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-26


Initialising variables:   0%|          | 0/217 [00:00<?, ?it/s]

BUILDING CONSTRAINTS:   0%|          | 0/8 [00:00<?, ?it/s]

c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:195: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tache_precedante = taches_df[taches_df["Ordre"] == ordre -
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:204: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  list_wagons = list(correspondances_df[correspondances_df["train_id"] ==
c:\Users\remgo\Desktop\ProjetST7\git\contraintes.py:207: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sillon_arr = correspondances_df[correspondances_df["id_wagon"] ==



Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 110306 rows, 151795 columns and 323249 nonzeros
Model fingerprint: 0x9c37303c
Model has 191140 general constraints
Variable types: 0 continuous, 151795 integer (112805 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+04]
  RHS range        [1e+00, 2e+06]
  GenCon rhs range [5e-01, 2e+01]
  GenCon coe range [1e+00, 2e+01]
Presolve removed 335331 rows and 381275 columns (presolve time = 5s) ...
Presolve removed 335331 rows and 381275 columns (presolve time = 10s) ...
Presolve removed 80655 rows and 139890 columns
Presolve time: 11.82s
Presolved: 29651 rows, 11905 columns, 90158 nonzeros
Variable types: 0 continuous, 11905 integer (11562 binary)
Found heuristic solution:

KeyboardInterrupt: 

Exception ignored in: 'gurobipy.logcallbackstub'
Traceback (most recent call last):
  File "c:\Users\remgo\anaconda3\envs\st7\lib\site-packages\ipykernel\iostream.py", line 535, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]
KeyboardInterrupt: 


 1638288  8160 10100.0000   77   56 10340.0000 10100.0000  2.32%   124 11705s


Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

